In [6]:
# Common imports
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal

# Networking/packet analysis
import pyshark
import scapy.all as scapy
import networkx as nx
from ipywidgets import interact, Dropdown


# Helper: inline plots
%matplotlib inline


# App Layer Scope Map — Packet Visibility

**Goal:** Visualize broadcast, multicast, unicast flows.


In [7]:
# small network graph
G = nx.Graph()
edges = [
    ("HostA","Switch1"), ("HostB","Switch1"), ("Router","Switch1"),
    ("Router","Switch2"), ("HostC","Switch2"), ("HostD","Switch2")
]

G.add_edges_from(edges)
pos = nx.spring_layout(G, seed=42)

In [8]:
from ipywidgets import Dropdown


def scope_map(mode="unicast"):
    plt.figure(figsize=(6,4))
    colors = []
    if mode == "unicast":
        src, dst = "HostA","HostC"
        nx.draw(G, pos, with_labels=True, node_color="lightgray")
        nx.draw_networkx_edges(G, pos, edgelist=[("HostA","Switch1"),("Switch1","Router"),("Router","Switch2"),("Switch2","HostC")],
                               edge_color="blue", width=3)
        plt.title("Unicast: direct path HostA → HostC")
    elif mode == "broadcast":
        nx.draw(G, pos, with_labels=True, node_color="mistyrose")
        plt.title("Broadcast: frame flooded to all nodes within domain")
    elif mode == "multicast":
        members = ["HostA","HostC","HostD"]
        color_map = ["lightgreen" if n in members else "lightgray" for n in G]
        nx.draw(G, pos, with_labels=True, node_color=color_map)
        plt.title("Multicast: only subscribed members receive")
    plt.axis("off"); plt.show()

interact(scope_map, mode=Dropdown(options=["unicast","broadcast","multicast"], value="unicast"))


interactive(children=(Dropdown(description='mode', options=('unicast', 'broadcast', 'multicast'), value='unica…

<function __main__.scope_map(mode='unicast')>

### Application-Layer View
| Mode | Who receives | Typical example | Protocols |
|------|---------------|-----------------|------------|
| Unicast | one peer | HTTP request | TCP |
| Broadcast | everyone on link | ARP, DHCP-DISCOVER | UDP |
| Multicast | selected group | IPTV, group chat | UDP + IGMP |

These scopes sit *on top of* the same physical medium you analyzed before:
- **Reflections** (WireScope) limit how far the wave can go.
- **Collisions** (CSMA/CD) decide who speaks when.
- **RTT** (TCP lab) controls latency at session setup.
- At the application layer, these constraints define *reachability* and *performance.*


In [9]:
import random, time

def simulate_flood(start="HostA", mode="broadcast"):
    reached = set()
    frontier = [start]
    steps = []
    while frontier:
        next_frontier=[]
        for node in frontier:
            reached.add(node)
            for n in G.neighbors(node):
                if n not in reached:
                    next_frontier.append(n)
        frontier=next_frontier
        steps.append(reached.copy())
    return steps

steps = simulate_flood()
for i,s in enumerate(steps,1):
    print(f"Step {i}: {sorted(list(s))}")


Step 1: ['HostA']
Step 2: ['HostA', 'Switch1']
Step 3: ['HostA', 'HostB', 'Router', 'Switch1']
Step 4: ['HostA', 'HostB', 'Router', 'Switch1', 'Switch2']
Step 5: ['HostA', 'HostB', 'HostC', 'HostD', 'Router', 'Switch1', 'Switch2']


### From Physical to Application Scope

| Layer | “Scope” Meaning | Constraint Source |
|--------|-----------------|-------------------|
| Physical | Distance cable can carry signal | attenuation, reflections |
| Data Link | Devices sharing one collision domain | propagation delay |
| Network | Devices sharing one IP subnet | router separation |
| Transport | End-to-end pair connection | RTT, congestion |
| Application | Who receives message or session | unicast/broadcast/multicast rules |

**Observation:** every upper-layer scope is *bounded* by the lower-layer physics.


1. Why can’t broadcast traffic cross routers by default?
2. What physical-layer limits still apply to application traffic?
3. How does multicast efficiency depend on proper lower-layer timing?
4. Give a real-world case where excessive broadcast load (ARP storms, DHCP) hurts app performance.


**Summary — “From Waves to Web”**

- A web request’s unicast packet rides on a TCP session (Lab 05).
- That TCP session rides on frames sharing a medium governed by CSMA/CD (Lab 04).
- Those frames propagate along cables whose reflections you analyzed (Lab 02).
- And their electrical signals obey Nyquist’s limits (Lab 01).

At the top, the application’s **scope**—who hears the message—is the human-visible effect of every physical decision beneath it.
